### Setup Ultralytics and put image data in snowflake stage

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import shutil
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
! pip install ultralytics

In [ ]:
from ultralytics import YOLO

In [ ]:
try:
    shutil.rmtree('/home/app/yoloruns')
except:
    pass

In [ ]:
!mkdir -p /home/app/yoloruns
!yolo settings runs_dir=/home/app/yoloruns

In [ ]:
# Train yolo11n on VOC for 2 epochs
!yolo train model=yolo11n.pt data=VOC.yaml epochs=2 imgsz=640 batch=32 device=0

In [ ]:
!yolo val model=/home/app/yoloruns/detect/train/weights/best.pt data=VOC.yaml

In [ ]:
create stage if not exists MODELREGISTRYTOSPCSYOLO_INTERNALSTAGE ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE') DIRECTORY = (ENABLE = TRUE);

In [ ]:
!ls /home/app/yoloruns/detect/train/weights/

In [ ]:
session.file.put("/home/app/yoloruns/detect/train/weights/best.pt", "@MODELREGISTRYTOSPCSYOLO_INTERNALSTAGE/model/", auto_compress=False, overwrite=True, parallel=99)

In [ ]:
ls @MODELREGISTRYTOSPCSYOLO_INTERNALSTAGE/model/

In [ ]:
session.file.put(f"/home/app/datasets/VOC/images/test2007/*.jpg", "@MODELREGISTRYTOSPCSYOLO_INTERNALSTAGE/data/", auto_compress=False, overwrite=True, parallel=99)

In [ ]:
ls @MODELREGISTRYTOSPCSYOLO_INTERNALSTAGE/data/